In [14]:
step = int(16)
input_video = "data/chair021.mp4"
output_video = "data/output.mp4"

In [42]:
import cv2 as cv
import inaccel.coral as inaccel
import numpy as np
from IPython.display import Video, HTML
from tqdm import tqdm
import math
from matplotlib import pyplot as plt

In [48]:
def play_vid(vid_list):
    return HTML('<table><tr>' +
                ''.join(['<td><video alt="test" controls><source src=' + vid + ' type="video/mp4"></video></td>'
                         for vid in vid_list]) +
                '</tr></table>'
                )

In [53]:
play_vid([input_video])

""


In [54]:
cameraMA_l = []
cameraMA_r = []
distC_l = []
distC_r = []
irA_l = []
irA_r = []
bm_state_arr = []
for j in range(step):
    cameraMA_l.append(inaccel.array([933.173, 0.0, 663.451, 0.0, 933.173, 377.015, 0.0, 0.0, 1.0], dtype=np.float32))
    cameraMA_r.append(inaccel.array([933.467, 0.0, 678.297, 0.0, 933.467, 359.623, 0.0, 0.0, 1.0], dtype=np.float32))

    distC_l.append(inaccel.array([-0.169398, 0.0227329, 0.0, 0.0, 0.0], dtype=np.float32))
    distC_r.append(inaccel.array([-0.170581, 0.0249444, 0.0, 0.0, 0.0], dtype=np.float32))

    irA_l.append(inaccel.array([0.0011976323, -0.0000000019, -0.8153011732, 0.0000000007, 0.0011976994, \
                               -0.4422348617,  0.0000126839,  0.0000001064, 0.9913820905], dtype=np.float32))
    irA_r.append(inaccel.array([0.0011976994,  0.0000000000, -0.8047567905, -0.0000000000, 0.0011976994, \
                               -0.4420566166, -0.0000000000, -0.0000001064,  1.0000392898], dtype=np.float32))

    bm_state_arr.append(inaccel.array([0, 15, 31, 15, 0, 48, 20, 15, 16, 3, 0], dtype=np.int32))


In [55]:
vid = cv.VideoCapture(input_video)
if (vid.isOpened()== False): 
      print("Error opening video stream or file")

frame_num = int(vid.get(cv.CAP_PROP_FRAME_COUNT))
height = int(vid.get(cv.CAP_PROP_FRAME_HEIGHT))
width  = int(vid.get(cv.CAP_PROP_FRAME_WIDTH)/2)
# fourcc  = int(vid.get(cv.CAP_PROP_FOURCC))

fourcc = cv.VideoWriter_fourcc(*'mp4v')
out = cv.VideoWriter(output_video, fourcc, 20.0, (height,width))

frames = []
sessions = []
left_mats = []
right_mats = []
disp_mats = []
for i in tqdm(range(math.ceil(frame_num/step))):
    frames.clear()
    sessions.clear()
    left_mats.clear()
    right_mats.clear()
    disp_mats.clear()
    
    for j in range(step):
        ret, frame = vid.read()
        if ret == False: 
            break
        else:
            frames.append(frame)

    for j in range(len(frames)):
        frame_left = cv.cvtColor(cv.resize(frames[j][:,:width,:], (1280, 720)), cv.COLOR_BGR2GRAY)
        frame_right = cv.cvtColor(cv.resize(frames[j][:,width:,:], (1280, 720)), cv.COLOR_BGR2GRAY)
            
        left_mats.append(inaccel.array(frame_left))
        right_mats.append(inaccel.array(frame_right))
        
        disp_mats.append(inaccel.ndarray((720, 1280), dtype=np.uint16))

        req = inaccel.request('com.xilinx.vitis.vision.stereoBM')
        req.arg(left_mats[j])
        req.arg(right_mats[j])
        req.arg(disp_mats[j])
        req.arg(cameraMA_l[j])
        req.arg(cameraMA_r[j])
        req.arg(distC_l[j])
        req.arg(distC_r[j])
        req.arg(irA_l[j])
        req.arg(irA_r[j])
        req.arg(bm_state_arr[j])
        req.arg(np.int32(720))
        req.arg(np.int32(1280))
        sessions.append(inaccel.submit(req))
            
    for j in range(len(sessions)):
        inaccel.wait(sessions[j])
        disp_img = (disp_mats[j].view(np.ndarray)*(256.0 / 48.0) / (16.0)).astype(np.uint8)
        disp_img = cv.resize(cv.cvtColor(disp_img, cv.COLOR_GRAY2BGR), (height,width))
        out.write(disp_img)
vid.release()
out.release()

100%|██████████| 112/112 [00:23<00:00,  4.68it/s]


In [52]:
play_vid([input_video, output_video])

,


In [57]:
!jupyter nbextension install @jupyter-widgets/jupyterlab-manager jupyter-webrtc

Traceback (most recent call last):
  File "/usr/local/bin/jupyter-nbextension", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/site-packages/jupyter_core/application.py", line 268, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/usr/local/lib/python3.7/site-packages/notebook/nbextensions.py", line 983, in start
    super(NBExtensionApp, self).start()
  File "/usr/local/lib/python3.7/site-packages/jupyter_core/application.py", line 257, in start
    self.subapp.start()
  File "/usr/local/lib/python3.7/site-packages/notebook/nbextensions.py", line 711, in start
    self.install_extensions()
  File "/usr/local/lib/python3.7/site-packages/notebook/nbextensions.py", line 672, in install_extensions
    raise ValueError("Only one nbextension allowed at a time. "
ValueError: Only one nbextensi

In [79]:
!/home/jovyan/.local/bin/jupyter-labextension install @jupyter-widgets/jupyterlab-manager jupyter-webrtc

An error occured.
ValueError: Please install nodejs 5+ and npm before continuing installation. nodejs may be installed using conda or directly from the nodejs website.
See the log file for details:  /tmp/jupyterlab-debug-_qwmg6w0.log


In [80]:
!conda install nodejs

/bin/sh: 1: conda: not found


In [75]:
!jupyter contrib nbextension install --user
!jupyter nbextension enable --py widgetsnbextension
!jupyter labextension install @jupyter-widgets/jupyterlab-manager@0.38

Traceback (most recent call last):
  File "/usr/local/bin/jupyter", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/site-packages/jupyter_core/command.py", line 247, in main
    command = _jupyter_abspath(subcommand)
  File "/usr/local/lib/python3.7/site-packages/jupyter_core/command.py", line 134, in _jupyter_abspath
    'Jupyter command `{}` not found.'.format(jupyter_subcommand)
Exception: Jupyter command `jupyter-contrib` not found.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Traceback (most recent call last):
  File "/usr/local/bin/jupyter", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/site-packages/jupyter_core/command.py", line 247, in main
    command = _jupyter_abspath(subcommand)
  File "/usr/local/lib/python3.7/site-packages/jupyter_core/command.py", line 134, in _jupyter_abspath
    'Jupyter command `{}` not found.'.format(jupyter_subcommand)
Exception: Jupyter command `jupyter-lab

In [83]:
!/home/jovyan/.local/bin/jupyter-labextension install jupyterlab_iframe
!jupyter serverextension enable --py jupyterlab_iframe

An error occured.
ValueError: Please install nodejs 5+ and npm before continuing installation. nodejs may be installed using conda or directly from the nodejs website.
See the log file for details:  /tmp/jupyterlab-debug-ckqq2zi_.log
Enabling: jupyterlab_iframe.extension
- Writing config: /home/jovyan/.jupyter
    - Validating...
      jupyterlab_iframe.extension  OK
